In [ ]:
# Setup
path = './data/hmdb51'
path_rowframes = './data/hmdb51/rawframes/'
path_annotations = './data/hmdb51/annotations/'

img_height = 224
img_width = 224
batch_size = 32
num_classes = 51

In [ ]:
num_frames_desired = 10     #number of frames per clip
type_frame = 'img'          #img / flow_x / flow_y
partition = 'train'